In [15]:
%load_ext autoreload
%autoreload 2

In [16]:
import os, sys
from pathlib import Path
sys.path += [str(Path(os.getcwd()).parent.parent / "model_training")]

In [ ]:
# config

# how long dose it take to build a 'image_index' 
# -> lode images -> make vectors    --> SSIndex 
#                   make meta table _/

# draw train, val and test datasets

In [17]:
root_logo_datasetpath = "/home/schulz-kees/data/datasets/logo_dataset/data"

In [4]:
# load raw meta data
from glob import glob
import pandas as pd
from siuba import _, group_by, summarize, mutate


def load_dataset(root_path):
    df = pd.concat([pd.read_parquet(x) for x in glob(f"{root_path}/*/*.parquet")]).reset_index(drop=True)
    return (df >> mutate(path= _.class_id + "/" + _.image_id))
#logo_image_df = load_dataset(root_logo_datasetpath)

logo_class_df = pd.read_parquet(glob(f"{root_logo_datasetpath}/*.parquet")[0]).reset_index(drop=True)[["class_id", "logonummer","variant", "name", "length", "width","height","volume_bottles", "num_bottles","description"]]


#logo_image_df.to_parquet("./data/logo_image_df.parquet")
#logo_class_df.to_parquet("./data/logo_class_df.parquet")

In [ ]:
# split train test val
from model_training.lib.keesds import get_all_values_geq_n, split, split_by_col_values
from siuba import filter as _filter, summarize, _

# i don't know a good name.. 
def gen_train_val_test(df):
    values = get_all_values_geq_n(df, "class_id", 15)
    df_more_then_15, rest1 = split_by_col_values(df, "class_id", values)

    train_df, rest2 = split(df_more_then_15, "class_id", 10)
    validate_df, rest3 = split(rest2, "class_id", 2)
    test_df, _  = split(rest3, "class_id", 3)

    values = get_all_values_geq_n(rest1, "class_id", 3)
    df_more_then_3, _ = split_by_col_values(rest1, "class_id", values)

    test_df_unknown, _  = split(df_more_then_3, "class_id", 3)

    return train_df, validate_df, pd.concat([test_df, test_df_unknown]).reset_index(drop=True)


 
train_df, validate_df, test_df = gen_train_val_test(logo_image_df)
#train_df.to_parquet("./data/logo_train.parquet")
#validate_df.to_parquet("./data/logo_validate.parquet")
#test_df.to_parquet("./data/logo_test.parquet")

In [23]:
import pandas as pd 

logo_image_df = pd.read_parquet("./data/logo_image_df.parquet")
logo_class_df = pd.read_parquet("./data/logo_class_df.parquet")

train_df = pd.read_parquet("./data/logo_train.parquet")
validate_df = pd.read_parquet("./data/logo_validate.parquet")
test_df = pd.read_parquet("./data/logo_test.parquet")


if any(train_df["image_id"].isin(validate_df["image_id"])):
    print("1 not allwed")

if any(train_df["image_id"].isin(test_df["image_id"])):
    print("2 not allwed")

if any(test_df["image_id"].isin(validate_df["image_id"])):
    print("3 not allwed")

# missing values!

In [19]:
from model_training.lib.keesds import fillna_by_col, combin_cols

from siuba import _, mutate, filter as _filter
from siuba.siu import symbolic_dispatch
from typing import List
import math
import pandas as pd

@symbolic_dispatch
def get_geometry(a, b):
    def inner(a,b):
        if math.isnan(a):
            return b
        else:
            return a
    return [inner(x,y) for x,y in zip(a, b)]


def fill_missing_values(df, cols, cols_alternative, label = "class_id"):
    for col, a_col in zip(cols, cols_alternative):
        fillna_by_col(df, "class_id", col)
        df[col] = get_geometry(df[col], df[a_col])
    
    return df >> mutate(geometry=combin_cols(*[df[x] for x in cols]))


# clean train df...
def clean_df(df) -> pd.DataFrame:
    df = pd.merge(df, logo_class_df, on="class_id", how="left")
    df_clean = fill_missing_values(df, ['kastenbreite','kastenhoehe', 'kastenlaenge'],
                            ['width', 'height', 'length'])
    return (df_clean >> _filter(~_.kastenlaenge.isnull())).reset_index(drop=True)

In [25]:
extra_classes_path = "/home/schulz-kees/data/github/model_training/notebooks/models/data/logo_add_extra_classes.parquet"


clean_df(pd.read_parquet(extra_classes_path)).to_parquet("/home/schulz-kees/data/github/model_training/notebooks/models/data/logo_add_extra_classes_clean.parquet")


All-NaN slice encountered


In [15]:
#print(len(validate_df))
#df_train_clean = clean_df(train_df)
#df_train_clean.to_parquet("./data/train_test_df.parquet")
clean_df(validate_df)[:50].to_parquet("./data/val_test_df_klein.parquet")

/home/schulz-kees/.cache/pypoetry/virtualenvs/model-training-L-UrGKZC-py3.8/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1119: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(a, func=_nanmedian, axis=axis, out=out,


In [88]:
#clean_df(validate_df).to_parquet("./data/val_test_df.parquet")

All-NaN slice encountered


# Azure run

In [12]:
# parameter
EXPERIMENT_NAME = "test_"
COMPUTE_TARGET = "rohan"#"rohan" #glamdring #mordor

In [13]:
from azureml.core import Workspace, ScriptRunConfig, Environment
from azureml.core.conda_dependencies import CondaDependencies

# get workspace
ws = Workspace.from_config()

# get data from workspace ("folder")
ds = ws.get_default_datastore()
root_data_path = ds.path('logo_dataset_v1-beta/data/').as_mount()
train_dataset_path = ds.path("logo_tmp/train_test_df.parquet").as_mount()
validate_dataset_path = ds.path("logo_tmp/val_test_df.parquet").as_mount()



# config environment (https://docs.microsoft.com/en-us/azure/machine-learning/resource-curated-environments)
env = Environment.get(ws, name="gpu_test_env") 
# https://github.com/Azure/MachineLearningNotebooks/issues/1483
from azureml.core.runconfig import DockerConfiguration
dc = DockerConfiguration(use_docker=True, shm_size='30g')

# config script run
s1 = ScriptRunConfig(
    source_directory='../../src/',
    script='./scripts/train_script.py',
    compute_target=COMPUTE_TARGET,
    environment=env,
    docker_runtime_config=dc,
    arguments = [
        '--root_data_path', str(root_data_path),
        '--train_dataset_path', str(train_dataset_path),
        '--validation_dataset_path', str(validate_dataset_path),
        '--max_epochs', 600,   # number of max epochs...
        '--num_samples_epoch', 250, # number of triplets in 1 epoch 
        '--interval_update_index', 16, # every x epochs the index gets updated
        '--batch_size', 15, # test to set it to 15?
        '--num_workers', 4,
        '--gamma', 0.05,
        '--ss_index_name', "HD_fdcc2c00-9114-4106-ae69-2f38620f3f5c_4",
        '--ss_index_version', 37,
        '--model_name', "HD_fdcc2c00-9114-4106-ae69-2f38620f3f5c_4.ptm",
        '--model_version', 37,
        '--max_loss', 100,
        '--freeze', 17
    ]
)


# https://azure.github.io/azureml-cheatsheets/docs/cheatsheets/python/v1/data/
s1.run_config.data_references[root_data_path.data_reference_name] = root_data_path.to_config()
s1.run_config.data_references[train_dataset_path.data_reference_name] = train_dataset_path.to_config()
s1.run_config.data_references[validate_dataset_path.data_reference_name] = validate_dataset_path.to_config()

In [14]:
# hypterdive
from azureml.train.hyperdrive import normal, uniform, RandomParameterSampling, PrimaryMetricGoal
from azureml.train.hyperdrive import MedianStoppingPolicy
from azureml.train.hyperdrive import HyperDriveConfig

param_space = {
                 '--lr': uniform(0.0005, 0.0025),
                 '--margin': uniform(5, 15),
                 '--loss_weight': uniform(0.30, 0.80)
              }

param_sampling = RandomParameterSampling(param_space)

#https://docs.microsoft.com/en-us/azure/machine-learning/how-to-tune-hyperparameters
early_termination_policy = MedianStoppingPolicy(evaluation_interval=1, delay_evaluation=5)

hd_config = HyperDriveConfig(run_config=s1,
                             hyperparameter_sampling=param_sampling,
                             policy=None,
                             primary_metric_name="class_acc",
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs=8,
                             max_concurrent_runs=8)

In [15]:
# run experiment

from azureml.core.experiment import Experiment
#from azureml.pipeline.core.pipeline import Pipeline
# Copy src to a tmp src for Azure
#

experiment = Experiment(ws, EXPERIMENT_NAME)
# Build the Pipeline from the Steps 
#steps = [s1_hypesr_train]

#pipeline = Pipeline(workspace=ws, steps=steps)
# Run the Pipeline
#pipeline_run = experiment.submit(s1)
pipeline_run = experiment.submit(hd_config)

In [ ]:
#pipeline_run.wait_for_completion()

# cuda memory

In [ ]:
#https://stackoverflow.com/questions/59129812/how-to-avoid-cuda-out-of-memory-in-pytorch
import gc
#del variables
gc.collect()
import torch
torch.cuda.empty_cache()

# https://deeptechtalker.medium.com/pytorch-cant-allocate-more-memory-1c36d7c9df4
# https://betterprogramming.pub/how-to-make-your-pytorch-code-run-faster-93079f3c1f7b

In [16]:
from model_training._script_util import get_children
from model_training._model import LogoModel, get_resnet18

get_children(LogoModel(get_resnet18(cut=1)))[-17:]

ModuleNotFoundError: No module named '_model'

## Add private python package

In [20]:
#https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-private-python-packages

#whl_url = Environment.add_private_pip_wheel(workspace=ws,file_path = "../../libs/indexmodel-0.3.1-py3-none-any.whl")
myenv = Environment(name="gpu_test_env")

In [22]:
myenv.get

{'imageExistsInRegistry': False, 'intellectualPropertyPublisher': None, 'imageCapabilities': {'canAccessData': True}, 'ingredients': {'dockerfile': 'FROM mcr.microsoft.com/azureml/openmpi4.1.0-cuda11.1-cudnn8-ubuntu18.04:20211111.v1\n\nENV AZUREML_CONDA_ENVIRONMENT_PATH /azureml-envs/pytorch-1.10\n\n# Create conda environment\nRUN conda create -p $AZUREML_CONDA_ENVIRONMENT_PATH \\\n    python=3.8 \\\n    pip=20.2.4 \\\n    pytorch=1.10.0 \\\n    torchvision=0.11.1 \\\n    cudatoolkit=11.1.1 \\\n    nvidia-apex=0.1.0 \\\n    gxx_linux-64 \\\n    -c anaconda -c pytorch -c conda-forge\n\n# Prepend path to AzureML conda environment\nENV PATH $AZUREML_CONDA_ENVIRONMENT_PATH/bin:$PATH\n\n# Install pip dependencies\nRUN pip install \'matplotlib>=3.3,<3.4\' \\\n                \'psutil>=5.8,<5.9\' \\\n                \'tqdm>=4.59,<4.63\' \\\n                \'pandas>=1.3,<1.4\' \\\n                \'scipy>=1.5,<1.8\' \\\n                \'numpy>=1.10,<1.22\' \\\n                \'azureml-core=

In [18]:
whl_url

'https://shire5329111891.blob.core.windows.net/azureml/Environment/azureml-private-packages/indexmodel-0.3.1-py3-none-any.whl'